In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
pts = 30000
mdat = pd.read_csv('cleanCMURevenueAdjusted.tsv', delimiter='\t',encoding='utf-8-sig')
plotsum = mdat[mdat.columns[7]]
mdat["Genres"] = mdat.apply(lambda row: eval(row["Genres"]), axis=1)
#g3 = mdat["Genres"]
gen = mdat["Genres"]
titles = mdat["Name"]
mdat.head()

#Split
Train2, Test2 = train_test_split(mdat, test_size=0.3)
#Get plot data
TrainPlot = Train2[mdat.columns[7]]
TestPlot = Test2[mdat.columns[7]]
#Get genre data
TrainGen = Train2["Genres"]
TestGen = Test2["Genres"]

In [8]:
#Tf-idf the training plots
TrainVec = TfidfVectorizer(max_features = 3000, stop_words = stopwords.words('english'))
TrainPlotsVec = TrainVec.fit_transform(TrainPlot)
#fit the test plots to this tf-idf
TestPlotsVec =  TrainVec.transform(TestPlot)

In [9]:
#Find the points with genre lists that include comedy
comTest = np.zeros((len(TestGen),))
for jj in range(len(TestGen)):
    if any(x == 'Comedy' for x in TestGen.iloc[jj]):
        comTest[jj] = 1
        
comTrain = np.zeros((len(TrainGen),))
for jj in range(len(TrainGen)):
    if any(x == 'Comedy' for x in TrainGen.iloc[jj]):
        comTrain[jj] = 1

In [10]:
#Find the points with genre lists that include horror or thriller
horTrain = np.zeros((len(TrainGen),))
for jj in range(len(TrainGen)):
    if any(x == 'Horror' or x == 'Thriller' for x in TrainGen.iloc[jj]):
        horTrain[jj] = 1
        
horTest = np.zeros((len(TestGen),))
for jj in range(len(TestGen)):
    if any(x == 'Horror' or x == 'Thriller' for x in TestGen.iloc[jj]):
        horTest[jj] = 1

In [11]:
#GBC classify comedy
abc = GradientBoostingClassifier()#n_estimators = 5, learning_rate = 1.0, max_depth = 1, random_state = 0 )
m = abc.fit(TrainPlotsVec , comTrain)
score = m.score(TestPlotsVec, comTest)
print(score)

0.7681250987205812


In [12]:
#% of points that are not comedy
1-sum(comTest)/len(comTest) 

0.7472753119570368

In [13]:
#GBC classify horror
abc = GradientBoostingClassifier()#n_estimators = 5, learning_rate = 1.0, max_depth = 1, random_state = 0 )
m = abc.fit(TrainPlotsVec , horTrain)
horscore = m.score(TestPlotsVec, horTest)
print(horscore)

0.8367556468172485


In [14]:
1-sum(horTest)/len(horTest) 

0.7820249565629442